In [1]:
## Import all the necessary Libraries required to run this program
import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup
import re

from nltk import tokenize

from fuzzywuzzy import fuzz

from tqdm import tqdm_notebook as tqdm

import operator

import spacy
from spacy import displacy

In [2]:
## The link for the Wikipedia page I webscraped - https://en.wikipedia.org/wiki/R2-D2 
## page_url = "https://en.wikipedia.org/wiki/R2-D2"

page_url = input("Enter from where you need to Extract the Article! : ")

Enter from where you need to Extract the Article! : https://en.wikipedia.org/wiki/R2-D2


In [3]:
## The HTTP 200 OK success status response code indicates that the request has succeeded.

page = requests.get(page_url)
print(page.status_code)


200


In [4]:
## Let us extract all the Text Data - to understand better Refer README file
soup = BeautifulSoup(page.text)
text_file = soup.find_all('p')

In [5]:
## Print an example to understand what we have got
text_file[0:3]

[<p class="mw-empty-elt">
 </p>,
 <p><b>R2-D2</b> (<span class="nowrap"><span class="IPA nopopups noexcerpt"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="/ˌ/: secondary stress follows">ˌ</span><span title="/ɑːr/: 'ar' in 'far'">ɑːr</span><span title="'t' in 'tie'">t</span><span title="/uː/: 'oo' in 'goose'">uː</span><span title="/ˈ/: primary stress follows">ˈ</span><span title="'d' in 'dye'">d</span><span title="/iː/: 'ee' in 'fleece'">iː</span><span title="'t' in 'tie'">t</span><span title="/uː/: 'oo' in 'goose'">uː</span></span>/</a></span></span>), or <b>Artoo-Detoo</b>,<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> is a fictional robot character in the <a href="/wiki/Star_Wars" title="Star Wars"><i>Star Wars</i> franchise</a> created by <a href="/wiki/George_Lucas" title="George Lucas">George Lucas</a>. A small <a class="mw-redirect" href="/wiki/Astromech_droid" title="Astromech droid">as

In [6]:
## Let us remove all the HTML Tags to obtain a much Clearer Text
def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', str(data))
    cleanr = re.compile(', ')
    cleantext = re.sub(cleanr, ',', str(cleantext))
    cleanr = re.compile('\'')
    cleantext = re.sub(cleanr, '', str(cleantext))
    cleanr = re.compile('\n,')
    cleantext = re.sub(cleanr, ' ', str(cleantext))
    
    return cleantext

In [7]:
data = striphtml(text_file)

In [8]:
## Print an example to understand the data at hand
print(data[0:3000])

[ R2-D2 (/ˌɑːrtuːˈdiːtuː/),or Artoo-Detoo,[1] is a fictional robot character in the Star Wars franchise created by George Lucas. A small astromech droid,R2-D2 is a major character and appears in nine out of the ten Star Wars films to date. Throughout the course of the films,R2 is a friend to C-3PO,Padmé Amidala,Anakin Skywalker,Leia Organa,Luke Skywalker,and Obi-Wan Kenobi in various points in the saga. English actor Kenny Baker played R2-D2 in all three original Star Wars films,and received billing credit for the character in the prequel trilogy,where Bakers role was reduced,as R2-D2 was portrayed mainly by radio controlled props and CGI models. In the sequel trilogy,Baker was credited as consultant for The Force Awakens; however,Jimmy Vee also co-performed the character in some scenes. Vee later took over the role beginning in The Last Jedi.[2] R2-D2s sounds and vocal effects were created by Ben Burtt. R2-D2 was designed in artwork by Ralph McQuarrie,co-developed by John Stears and b

In [9]:
sentences = tokenize.sent_tokenize(data)

In [10]:
len(sentences)

99

In [11]:
## Lets search for Questions based on Article for Answers
question = input("Enter any question related to the article: ")

Enter any question related to the article: When was R2-D2 inducted into the Robot Hall of Fame?


## Finding Context

In [12]:
## By searching through the article we can find where the context related to the Question is present

scores = []
index = []

for i in tqdm(range(len(sentences))) :
    score = fuzz.partial_ratio(question, sentences[i])
    scores.append(score)
    index.append(i)
    
dictionary_scores = dict(zip(index, scores))

probable_answer_index = max(dictionary_scores.items(), key=operator.itemgetter(1))[0]
score = dictionary_scores[probable_answer_index]

In [13]:
context = []

for i in range(probable_answer_index-2, probable_answer_index+2):
    context.append(sentences[i])

In [14]:
print(context)

['[32] Baker himself has said he probably only appears in footage caught while shooting the previous two movies.', '[33] For The Force Awakens,producer Kathleen Kennedy hired two fans,Lee Towarsey and Oliver Steeples,to build new R2-D2 robots for the film,after being impressed by their working replicas that were brought to Star Wars Celebration Europe in 2013.', '[34] R2-D2 was inducted into the Robot Hall of Fame in 2003.', 'The Smithsonian Institution included R2-D2 in its list of 101 Objects that Made America.']


In [15]:
## Lets use Spacy to find the Dependencies of each word in the sentence.
## Using the dependencies we can extract the a) Root word, b) Link between the subject and the verb c) Link between verb and Objects

imp_word = []

nlp = spacy.load("en")
doc = nlp(question)

for token in doc:
    if token.dep_ == 'ROOT':
        imp_word.append(token.text)
    elif token.dep_ == 'obj' or (token.dep_ == 'dobj') or (token.dep_ == 'iobj') or (token.dep_ == 'pobj'):
        imp_word.append(token.text)
    elif (token.dep_ == 'subj') or (token.dep_ == 'nsubj') or (token.dep_ == 'nsubjpass') or (token.dep_ == 'csubj') or (token.dep_ == 'csubjpass') :
        imp_word.append(token.text)
        
key_phrase = ' '.join(imp_word)

In [16]:
key_phrase

'inducted Hall Fame'

In [17]:
for sentence in context :
    score_2 = fuzz.token_set_ratio(sentence, key_phrase)
    
    if score_2 >= 50:
        print(sentence)
        print("Score : {}".format(score))

[34] R2-D2 was inducted into the Robot Hall of Fame in 2003.
Score : 83


## Test cases

In [18]:
def find_answer (sentences, question):
    
    ## By searching through the article we can find where the context related to the Question is present
    scores = []
    index = []

    for i in tqdm(range(len(sentences))) :
        score = fuzz.partial_ratio(question, sentences[i])
        scores.append(score)
        index.append(i)

    dictionary_scores = dict(zip(index, scores))

    probable_answer_index = max(dictionary_scores.items(), key=operator.itemgetter(1))[0]
    score = dictionary_scores[probable_answer_index]
    
    context = []
    
    for i in range(probable_answer_index-2, probable_answer_index+2):
        context.append(sentences[i])
    
    imp_word = []
    
    nlp = spacy.load("en")
    doc = nlp(question)
    
    for token in doc:
        if token.dep_ == 'ROOT':
            imp_word.append(token.text)
        elif token.dep_ == 'obj' or (token.dep_ == 'dobj') or (token.dep_ == 'iobj') or (token.dep_ == 'pobj'):
            imp_word.append(token.text)
        elif (token.dep_ == 'subj') or (token.dep_ == 'nsubj') or (token.dep_ == 'nsubjpass') or (token.dep_ == 'csubj') or (token.dep_ == 'csubjpass') :
            imp_word.append(token.text)
    
    key_phrase = ' '.join(imp_word)
    
    
    for sentence in context :
        score_2 = fuzz.token_set_ratio(sentence, key_phrase)
        
        if score_2 >= 50:
            print(sentence)
            print("Score : {}".format(score))

## Case - 1

In [20]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: who gave R2-D2's sounds and vocal effects?



[2] R2-D2s sounds and vocal effects were created by Ben Burtt.
Score : 76


## Case - 2

In [21]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: who was commissioned to fabricate the design of R2-D2?



[5]
Tony Dyson,owner of the special effects studio The White Horse Toy Company,was commissioned to fabricate the design,making four units operated by remote control.
Score : 78


## Case - 3

In [23]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: In Return of the Jedi what was the critical role of R2-D2?



In Return of the Jedi,R2-D2 plays a critical role in rescuing Luke,Leia,and Han from Jabba the Hutt.
Score : 71


## Case - 4

In [24]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Throughout the course of the films, who were R2-D2 friends?



Throughout the course of the films,R2 is a friend to C-3PO,Padmé Amidala,Anakin Skywalker,Leia Organa,Luke Skywalker,and Obi-Wan Kenobi in various points in the saga.
Score : 75


## Case - 5

In [25]:
question = input("Enter any question related to the article: ")
find_answer(sentences, question)

Enter any question related to the article: Throughout the course of the films, How many friends di R2-D2 have in total?



Throughout the course of the films,R2 is a friend to C-3PO,Padmé Amidala,Anakin Skywalker,Leia Organa,Luke Skywalker,and Obi-Wan Kenobi in various points in the saga.
Score : 66


### Note : This model is unable to do simple calculation like addition